<a href="https://colab.research.google.com/github/mananthavelu/data_and_statistical_modeling/blob/main/12_feed_forward_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Import the necessary libraries, dataset

In [1]:
# Buiding a NN model
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Doanloading the training ata from open datasets
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:07<00:00, 3.61MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 204kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.76MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.4MB/s]


In [3]:
# Doanload the test data from open datasets
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

In [4]:
# Define the batch size
batch_size = 64

# Create data loaders

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
# Creating models
# Accelerator usage
device = torch.accelerator.current_accelerator().type\
if torch.accelerator.is_available() else "cpu"

print("Device: {device}")

Device: {device}


In [17]:
# Define the model

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


**Optimizing the model parameters**

In [18]:
loss_function = nn.CrossEntropyLoss()# Define the loss function which will drive the optimization
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-3)#the optimizer which takes the loss information and updates the weight
# Loss function -> Gradients -> Weights adjustments

In [19]:
def train(dataloader, model, loss_function, optimizer):
  size = len(dataloader.dataset)# Size of the training dataset
  model.train()# Train the model
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    prediction = model(X)
    loss = loss_function(prediction, y)

    # Backpropagation
    loss.backward()# Calculate the gradients and attach it to .grad attribute for each parameter
    optimizer.step()# Optimizer has access to params # Update the weights of the model
    optimizer.zero_grad()# Reset the gradients to 0 so the gradients from prev. iteration dont accumulate

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1)* len(X)
      print(f"loss:{loss:>7f} [{current:>5d}/{size:.>5d}]")


In [24]:
def test(dataloader, model, loss_function):
  size = len(dataloader.dataset)
  num_of_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      predictions = model(X)
      test_loss += loss_function(predictions, y).item()
      correct += (predictions.argmax(1) == y). type(torch.float).sum().item()
  test_loss /= num_of_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100* correct):>0.1}%, Avg loss: {test_loss:>8f} \n")


In [25]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1} \n---------")
  train(train_dataloader, model, loss_function, optimizer)
  test(test_dataloader, model, loss_function)
print("Done")

Epoch 1 
---------
loss:1.904798 [   64/60000]
loss:1.877403 [ 6464/60000]
loss:1.753543 [12864/60000]
loss:1.784287 [19264/60000]
loss:1.689282 [25664/60000]
loss:1.638467 [32064/60000]
loss:1.649562 [38464/60000]
loss:1.566828 [44864/60000]
loss:1.579901 [51264/60000]
loss:1.456794 [57664/60000]
Test Error: 
 Accuracy: 6e+01%, Avg loss: 1.494772 

Epoch 2 
---------
loss:1.562737 [   64/60000]
loss:1.533797 [ 6464/60000]
loss:1.377496 [12864/60000]
loss:1.444535 [19264/60000]
loss:1.332216 [25664/60000]
loss:1.332405 [32064/60000]
loss:1.341581 [38464/60000]
loss:1.283259 [44864/60000]
loss:1.312234 [51264/60000]
loss:1.197712 [57664/60000]
Test Error: 
 Accuracy: 6e+01%, Avg loss: 1.237276 

Epoch 3 
---------
loss:1.312026 [   64/60000]
loss:1.302148 [ 6464/60000]
loss:1.128957 [12864/60000]
loss:1.233292 [19264/60000]
loss:1.105986 [25664/60000]
loss:1.138524 [32064/60000]
loss:1.157729 [38464/60000]
loss:1.112220 [44864/60000]
loss:1.148096 [51264/60000]
loss:1.048094 [57664/6000